In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)
print(com_data.shape)
com_data.head()

(224, 23)


,Team,Week,Result,Rec,Home,Opp,TmScore,OppScore,O_1stD,O_Tot_yd,...,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,D_TO,Offence,Defense,Sp.Tms,Prediction_LR,Prediction_ADA
0,Buffalo Bills,1,1,1-0,1,Chicago Bears,34.0,14.0,18.0,386.0,...,24.0,322.0,188.0,134.0,2.0,21.58,0.85,-0.52,0.418034,0.025844
1,Buffalo Bills,2,1,2-0,0,Indianapolis Colts,27.0,24.0,21.0,371.0,...,22.0,354.0,245.0,109.0,2.0,7.11,-1.29,-3.09,0.418034,0.025844
2,Buffalo Bills,3,1,3-0,1,Tampa Bay Buccaneers,34.0,24.0,22.0,407.0,...,27.0,446.0,411.0,35.0,1.0,20.56,-15.15,3.43,0.418034,0.025844
3,Buffalo Bills,4,0,3-1,1,Arizona Cardinals,20.0,37.0,24.0,401.0,...,27.0,465.0,249.0,216.0,1.0,8.94,-21.72,-2.64,0.418034,0.025844
4,Buffalo Bills,5,1,4-1,0,Seattle Seahawks,26.0,17.0,24.0,476.0,...,22.0,354.0,262.0,92.0,2.0,13.76,-6.71,-0.05,0.418034,0.025844


In [3]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
rf_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home','Prediction_LR','Prediction_ADA']]


# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

rf_model_data[season_stats] = rf_model_data[season_stats] * 16

#get indexs of every teams
team_index = com_data['Team']

rf_model_data = pd.get_dummies(rf_model_data)
rf_model_data.head()

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,TmScore,O_1stD,O_Tot_yd,O_P_Yd,O_R_Yd,O_TO,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,...,Opp_New Orleans Saints,Opp_New York Giants,Opp_New York Jets,Opp_Philadelphia Eagles,Opp_Pittsburgh Steelers,Opp_San Francisco 49ers,Opp_Seattle Seahawks,Opp_Tampa Bay Buccaneers,Opp_Tennessee Titans,Opp_Washington Football Team
0,34.0,288.0,6176.0,4992.0,1184.0,16.0,384.0,5152.0,3008.0,2144.0,...,0,0,0,0,0,0,0,0,0,0
1,27.0,336.0,5936.0,4320.0,1616.0,32.0,352.0,5664.0,3920.0,1744.0,...,0,0,0,0,0,0,0,0,0,0
2,34.0,352.0,6512.0,5296.0,1216.0,16.0,432.0,7136.0,6576.0,560.0,...,0,0,0,0,0,0,0,1,0,0
3,20.0,384.0,6416.0,4480.0,1936.0,32.0,432.0,7440.0,3984.0,3456.0,...,0,0,0,0,0,0,0,0,0,0
4,26.0,384.0,7616.0,5728.0,1888.0,16.0,352.0,5664.0,4192.0,1472.0,...,0,0,0,0,0,0,1,0,0,0


## MODELING

In [4]:
# Random forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Data processing
X = rf_model_data.drop('TmScore', axis=1).values
y = rf_model_data['TmScore'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10)

RF_model = RandomForestRegressor(n_estimators= 2000, random_state= 20)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


7.116543478260869


In [5]:
Pkl_RF_model = 'RF_model.sav'
pickle.dump(RF_model, open(Pkl_RF_model, 'wb'))

In [6]:
pd.options.display.max_rows = None

importance = RF_model.feature_importances_

feature_names = rf_model_data.drop('TmScore', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)

# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    team1_data = rf_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = rf_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(1,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) # added to address the NANs that was causing the error
    
    scores = RF_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner
    

# Potentially add for loop to insert the winner to next round (The names have to be in full names in score predicor) and add the seeding

### NFC Playoff Round 1 

In [7]:
scores, winner = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers')

Philadelphia Eagles will score 29.2
Tampa Bay Buccaneers will score 26.9
Philadelphia Eagles are the WINNERS!!!


In [7]:
scores, winner = Score_Predictor('San Francisco 49ers', 'Dallas Cowboys')

San Francisco 49ers will score 15.5
Dallas Cowboys will score 15.1
San Francisco 49ers are the WINNERS!!!


In [8]:
scores, winner = Score_Predictor('Arizona Cardinals', 'Los Angeles Rams')

Arizona Cardinals will score 15.6
Los Angeles Rams will score 15.3
Arizona Cardinals are the WINNERS!!!


### AFC Playoff Round 1 

In [9]:
scores, winner = Score_Predictor('Pittsburgh Steelers', 'Kansas City Chiefs')

Pittsburgh Steelers will score 15.1
Kansas City Chiefs will score 15.4
Kansas City Chiefs are the WINNERS!!!


In [10]:
scores, winner = Score_Predictor('Las Vegas Raiders', 'Cincinnati Bengals')

Las Vegas Raiders will score 15.4
Cincinnati Bengals will score 15.4
Las Vegas Raiders are the WINNERS!!!


In [11]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


### NFC Playoff Round 2

In [12]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


In [13]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


### NFC Playoff Round 2

In [14]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


In [15]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


### NFC Championship

In [16]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


### AFC Championship

In [17]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!


### Suberbowl

In [18]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

Buffalo Bills will score 15.5
New England Patriots will score 15.4
Buffalo Bills are the WINNERS!!!
